# The Task - Part of Speech Tagging

In this tutorial, we look at the structured prediction task of [Part Of Speech Tagging](https://en.wikipedia.org/wiki/Part-of-speech_tagging). This task consists of tagging/assigning the tokens in a sentence with Part of Speech (POS) tags/labels like Noun, Verb, Adjective, Adverb, Determiner, Pronoun, etcetera. 

We will,

1.   Download and explore a POS tagged corpus (dataset).
2.   Implement a couple of naive approaches.
3.   Understand Hidden Markov Model
4.   Understand Viterbi algorithm with an example.
5.   Compute transition probabilities and emission probabilities.
6.   Implement Viterbi algorithm.
7.   (Optional) Implement a RNN-based Tagging model

# 1. The Corpus

We will work with simple versions of the [Brown Corpus](https://en.wikipedia.org/wiki/Brown_Corpus) and [Penn Treebank Corpus](https://catalog.ldc.upenn.edu/docs/LDC95T7/cl93.html) which can be accessed using [NLTK](https://www.nltk.org/). 

**Let's begin by downloading the corpuses**

In [1]:
import nltk

# Downloading the corpuses
nltk.download('universal_tagset')
nltk.download('brown')
nltk.download('treebank')

from nltk.corpus import brown
from nltk.corpus import treebank

brown_corpus = list(brown.tagged_sents(tagset='universal'))
treebank_corpus = list(treebank.tagged_sents(tagset='universal'))

[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/louismanestar/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package brown to
[nltk_data]     /Users/louismanestar/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package treebank to
[nltk_data]     /Users/louismanestar/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


**Exploring the corpuses** 

The downloaded corpuses are a list of sentences. Each sentence is a list of tagged tokens. The tagged tokens are tuples of the form ```(token, tag)```. Each token and each tag is a string.

In [2]:
print('Size of brown_corpus is', len(brown_corpus), 'sentences\n')
print('Size of treebank_corpus is', len(treebank_corpus), 'sentences\n')
print('A sample from the brown_corpus:\n', brown_corpus[2])
print('\nA sample from the treebank_corpus:\n', treebank_corpus[3])

Size of brown_corpus is 57340 sentences

Size of treebank_corpus is 3914 sentences

A sample from the brown_corpus:
 [('The', 'DET'), ('September-October', 'NOUN'), ('term', 'NOUN'), ('jury', 'NOUN'), ('had', 'VERB'), ('been', 'VERB'), ('charged', 'VERB'), ('by', 'ADP'), ('Fulton', 'NOUN'), ('Superior', 'ADJ'), ('Court', 'NOUN'), ('Judge', 'NOUN'), ('Durwood', 'NOUN'), ('Pye', 'NOUN'), ('to', 'PRT'), ('investigate', 'VERB'), ('reports', 'NOUN'), ('of', 'ADP'), ('possible', 'ADJ'), ('``', '.'), ('irregularities', 'NOUN'), ("''", '.'), ('in', 'ADP'), ('the', 'DET'), ('hard-fought', 'ADJ'), ('primary', 'NOUN'), ('which', 'DET'), ('was', 'VERB'), ('won', 'VERB'), ('by', 'ADP'), ('Mayor-nominate', 'NOUN'), ('Ivan', 'NOUN'), ('Allen', 'NOUN'), ('Jr.', 'NOUN'), ('.', '.')]

A sample from the treebank_corpus:
 [('A', 'DET'), ('form', 'NOUN'), ('of', 'ADP'), ('asbestos', 'NOUN'), ('once', 'ADV'), ('used', 'VERB'), ('*', 'X'), ('*', 'X'), ('to', 'PRT'), ('make', 'VERB'), ('Kent', 'NOUN'), ('ciga

Note: The tokens in the corpuses are not pre-processed and we will continue for now without pre-processing.

**Exploring POS tags** 

**Q1:** How many unique POS tags, also called the tagset, do we have in these corpuses? Save the unique POS tags in a list called ```tagset```.

**Q2:** Enumerate all the POS tags and its frequencies/counts. Save it in a python dictionary ```count_tags_in_brown``` for the brown corpus and a python dictionary ```count_tags_in_treebank``` for the treebank corpus. (Sanity check - ```count_tags_in_brown['NUM']``` should be ```14874```) 

**Q3:** Which POS tag occurs most frequently and which POS tag occurs least frequently in both the corpuses?

In [3]:
import operator

##########
# For brown, computing count_tags_in_brown
count_tags_in_brown = {}
## ENTER CODE: Populate this dictionary in here

##########

# Tagset
tagset = count_tags_in_brown.keys()

##########
# For treebank, computing count_tags_in_treebank
count_tags_in_treebank = {}
## ENTER CODE: Populate this dictionary in here

##########

##########
## ENTER CODE: Find the most frequent and the least frequent POS Tags in here

###########

**Splitting into Training and Test sets**

We randomly sample 4000 sentences from the brown corpus as the ```test_set```. The rest of the brown corpus forms the ```train_set```. We will use the treebank corpus as the ```external_test_set```. 

Note: We could have tried other kinds of train-test splits too like 80-20 splits or mix the external test set with the brown test set, etcetera.

Also, for convenience, we separate the tokens and the tags out from the tuples ```(token, tag)``` to create two separate sets of ```train_set_X``` and ```train_set_Y``` (X for tokens only, Y for POS tags only). We do the same with the ```test_set``` and the ```external_test_set```.

In [4]:
import random

# For consistency of results everytime you run this cell
copy_of_brown = brown_corpus.copy() 
random.seed(20)                

# Splitting into train and test and external_test sets
random.shuffle(copy_of_brown)
train_set = copy_of_brown[4000:]
test_set = copy_of_brown[:4000]  # 4000 samples in the test set
external_test_set = treebank_corpus

# Separating tokens and tags
def _split_into_tokens_and_tags(given_set):
  """ Separating tokens and tags out from a list of list of (token, tag) tuples

  Arguments:
    given_set = a list of list of (token, tag) tuples. token and tag are strings

  Returns:
    token_set = a list of list of tokens only. order and shape is maintained.  
    tag_set = a list of list of tags only. order and shape is maintained.
  """
  token_set = given_set.copy() 
  tag_set = given_set.copy() 

  for i in range(len(given_set)):
    (token_only, tag_only) = zip(*given_set[i])
    token_set[i] = list(token_only)  # 0th element of tuple is token 
    tag_set[i] = list(tag_only)      # 1st element of tuple is POS tag
  
  return (token_set, tag_set)

# Creating x (sentences) and Y (tags) splits
(train_set_x, train_set_Y) = _split_into_tokens_and_tags(train_set)
(test_set_x, test_set_Y) = _split_into_tokens_and_tags(test_set)
(external_test_set_x, external_test_set_Y) \
= _split_into_tokens_and_tags(external_test_set)

# Example of how *_x and *_Y look like
# * could be train_set or test_set or external_test_set
print(test_set_x) # List of sentences where each sentence is a list of tokens
print(test_set_Y) # List of list of tags of the same shape as test_set_x

[['Ninety-nine', 'times', 'out', 'of', 'a', 'hundred', 'the', 'motel', 'manager', "doesn't", 'check', 'the', 'license', 'number', 'on', 'the', 'plates', 'against', 'the', 'license', 'number', 'the', 'tenant', 'writes', 'out', '.'], ['The', 'big', 'question', 'is', 'whether', ',', 'in', 'the', 'name', 'of', 'a', 'restored', 'Chinese-Soviet', 'solidarity', ',', 'the', 'Chinese', 'will', 'choose', 'to', 'persuade', 'the', 'Albanians', 'to', 'present', 'their', 'humble', 'apologies', 'to', 'Khrushchev', '--', 'or', 'get', 'rid', 'of', 'Enver', 'Hoxa', '.'], ['Telephone', 'wires', 'from', 'Louis', "Dufresne's", 'house', 'in', 'East', 'Manchester', 'to', 'the', 'Dufresne', 'lumber', 'job', 'near', 'Bourn', 'Pond', 'were', 'up', 'about', '1895', '.'], ['By', 'September', '1940', 'the', 'Suite', 'had', 'developed', 'into', 'a', 'collection', 'of', 'six', 'songs', ',', '``', 'four', 'spirituals', ',', 'a', 'dream', ',', 'and', 'a', 'lullaby', "''", '.'], ['Miss', 'Xydis', 'was', 'best', 'when',

# 2. Naive approaches to POS tagging

One naive approach could be to tag all words/tokens with the most frequent POS tag found in the entire training set. We call this the ```_most_freq_tag_model_1```. In other words, if ```'NUM'``` is the most frequent POS tag in the training set then we tag every word/token in the test set to ```'NUM'```.  

A better approach could be to tag each word/token to its most frequent POS tag as seen in the training set. We call this the ```_most_freq_tag_model_2```. In other words, for a word like ```'the'```, its most frequent POS tag as seen in the training set is ```'DET'```, so the model will tag ```'the'``` to ```'DET'``` whenever it sees it in the test set.

In the following exercises we will build models for these approaches and also build a function to evaluate them.

**We begin with approach 1: _most_freq_tag_model_1**

In Q3, we had found the most frequent POS tag across the entire brown corpus (which included both the train and test sets).

**Q4:** Check if the most frequent POS tag found in Q3 remains the most frequent POS tag in the ```train_set``` as well? We do this to ensure our naive approach is not informed by the ```test_set```. Save this tag to a variable called ```most_freq_tag```. Also, similar to Q2, save all the counts of tags in a python dictionary called ```count_tag```. Sanity check: ```count_tag['VERB']``` should be 170196)

In [5]:
##########
count_tag = {}
##ENTER CODE: Populate this dictionary in here

##########

sorted_count_tag = sorted(count_tag.items(), key=operator.itemgetter(1))

most_freq_tag = sorted_count_tag[-1][0]
print('The most frequent tag is', most_freq_tag)
print(count_tag['VERB'])  # Sanity check

IndexError: list index out of range

In [ ]:
sorted_count_tag

**Q5:** Implement a naive model as a function ```_most_freq_tag_model_1(x)``` which tags every token in a given test set ```x``` with the most frequent POS tag found in Q4. Ensure the output of this function is of the same shape and size as the input so that it is easy to evaluate it later on. For example, the call ```_most_freq_tag_model_1(test_set_x)``` should return a list of tuples of most frequent POS tag of the same size and shape as ```test_set_Y```. 

In [ ]:
def _most_freq_tag_model_1(x):
  """Tags every token with the most frequent POS tag"""
  ## ENTER CODE HERE
  return Y

**Q6:** Write a function ```_evaluate``` to evaluate the naive approach by measuring the accuracy of its predictions. Very simply, the function will take two list of lists of tags (of same shape) as arguments and count the number of times the tags are the same and divide this number with total number of tags. Sanity check: When you evaluate the naive approach ```_evaluate(_most_freq_tag_model_1(test_set_x), test_set_Y)``` you should get 23.8% accuracy. What is the accuracy of this model on the external test set?

**Q6_optional:** You can extend the evaluation function to compute *precision* and *recall* and *F-measure* for each POS tag as a way to practice and understand these concepts. What is the precision, recall and F-measure for ```'NOUN'``` and any other tag?

In [ ]:
##########
def _evaluate(predicted_Y, true_Y):
  correct_tags = 0
  total_tags = 0
  ## ENTER CODE IN HERE TO UPDATE THE ABOVE VARIABLES
  
  Accuracy = float(correct_tags) / total_tags
  print('Accuracy =', Accuracy)
##########

_evaluate(_most_freq_tag_model_1(test_set_x), test_set_Y) # Sanity check

print('\nExternal test set:')
_evaluate(_most_freq_tag_model_1(external_test_set_x), external_test_set_Y)

**Now we build a model for approach 2: _most_freq_tag_model_2**

For each word/token in the vocabulary, we need to retrieve the most frequent POS tag of that word as seen in the training set. Hence, we need to keep a count of every unique (token, tag) pair in the training set. We can do this using dictionaries in python. This will also prove to be useful later on in the exercise.

**Q7:** Create a dictionary ```count_token_tag``` that stores the counts of every unique (token, tag) pair as seen in the ```train_set```. Sanity check: ```count_token_tag['the']['DET']``` should be 58337 and ```count_token_tag['the']['VERB']``` should return an error.

In [ ]:
##########
count_token_tag = {}
for i in range(len(train_set)):
  for j in range(len(train_set[i])):
    ## ENTER CODE IN HERE
    
##########

# Sanity check
print(count_token_tag['the']['DET'])
print(count_token_tag['the']['VERB'])

**Q8:** Create a function ```_most_freq_tag``` which retrieves the most frequent POS tag for a given token as seen in the ```count_token_tag```. Sanity check: ```_most_freq_tag('the')``` is ```'DET'``` and ```_most_freq_tag('play')``` is ```'VERB'```. What about unseen words? What about ```_most_freq_tag('onomatopoeia')```? What can we do about it? One simple solution is to tag it with the ```most_freq_tag``` found in Q4.

In [ ]:
def _most_freq_tag(_token):
  """Returns the most frequent POS tag (mfPOSt) of a _token as seen in count_token_tag"""
  if _token in count_token_tag.keys():
    ## ENTER CODE HERE

    return mfPOSt
  else:
    return most_freq_tag  # this is a global constant. See Q4. Can you see why we did it this way?

print(_most_freq_tag('the'))
print(_most_freq_tag('play'))
print(_most_freq_tag('onomatopoeia'))  # for unseen words

**Q9:** Implement a naive model as a function ```_most_freq_tag_model_2(x)``` which tags every token in a test set ```x``` with its most frequent POS tag found in Q8. Ensure the output is of the same shape as ```x``` so that it is easy to evaluate. Evaluate the output of this model. Sanity check: ```_evaluate(_most_freq_tag_model_2(test_set_x), test_set_Y)``` should be 94.795%. Evaluate on the external test set too. Does the accuracy differ? Can you explain why?

In [ ]:
def _most_freq_tag_model_2(x):
  """Tags every token with its most frequent POS tag"""
  ## ENTER CODE HERE: use _most_freq_tag() from Q8

  return Y

_evaluate(_most_freq_tag_model_2(test_set_x), test_set_Y)
_evaluate(_most_freq_tag_model_2(external_test_set_x), external_test_set_Y)

# 3. Hidden Markov Model (HMM)

Let $W = (w_1, w_2, ..., w_n)$ be a sequence of observed words/tokens.

Let $T = (t_1, t_2, ..., t_n)$ be a sequence of tags and $\mathbb{T}_n$ be the set of all possible such sequences of tags of length $n$.

**Q10:** How many such sequences of tags exist? In other words, whats the size of the set $\mathbb{T}_n$?





In [ ]:
## ENTER YOUR ANSWER HERE

Our aim is to find the most probable sequence of tags $\tilde{T}$ for the observed sequence of words $W$ such that the probability $P(\tilde{T}|W) \geq P(T|W)$ for all possible $T \in \mathbb{T}_n$.

In other words, $\tilde{T} = \underset{T \in \mathbb{T}_n}{\operatorname{argmax}}P(T|W)$

Now, to estimate $P(T|W)$ we use the [Bayes' Theorem](https://en.wikipedia.org/wiki/Bayes%27_theorem) followed by the [Chain Rule](https://en.wikipedia.org/wiki/Chain_rule_(probability)) followed by the Markov Assumption ($P(t_m|t_1, t_2, ..., t_{m-1}) \approx P(t_m|t_{m-1})$) to get the following formulation for the HMM: 

$\tilde{T} = \underset{(t_1, t_2, ..., t_n) \in \mathbb{T}_n}{\operatorname{argmax}}P(t_1)P(w_1|t_1)P(t_2|t_1)P(w_2|t_2)...P(t_{m}|t_{m-1})P(w_{m}|t_{m})...P(t_n|t_{n-1})P(w_n|t_n)$

**Q11:** How do you interpret $P(t_m|t_{m-1})$ and $P(w_m|t_m)$? (A frequentist interpretation). How will you estimate these probabilities from the ```train_set```? How do you interpret $P(t_1)$ and estimate it from the ```train_set```?

In [ ]:
## ENTER YOUR ANSWER HERE

# 4. Viterbi algorithm (manual pen-paper example)
Recall from Q10, the number of possible tag sequences (size of $\mathbb{T}_n$) is enormous, especially for larger n and larger POS tagsets, which makes the brute force algorithm extremely inefficient. To find the solution $\tilde{T}$ quickly, we use a dynamic programming algorithm called [Viterbi](https://en.wikipedia.org/wiki/Viterbi_algorithm).

In this section, we will use the Viterbi algorithm to tag the following sentence manually:

***time flies like an arrow***

We assume some pre-computed transition probabilities and emission probabilities as follows:

Initial probabilities:
*   $P($NOUN$|start) = 0.5$
*   $P($DET$|start) = 0.4$
*   $P($VERB$|start) = 0.1$

Transition probabilities:
*   $P($NOUN$|$DET$) = 1.0$
*   $P($NOUN$|$NOUN$) = 0.2$
*   $P($VERB$|$NOUN$) = 0.7$
*   $P($PREP$|$NOUN$) = 0.1$
*   $P($DET$|$VERB$) = 0.4$
*   $P($NOUN$|$VERB$) = 0.4$
*   $P($PREP$|$VERB$) = 0.1$
*   $P($VERB$|$VERB$) = 0.1$
*   $P($DET$|$PREP$) = 0.6$
*   $P($NOUN$|$PREP$) = 0.4$

Emission probabilities:
*   $P($time$|$NOUN$) = 0.7$
*   $P($time$|$VERB$) = 0.1$
*   $P($flies$|$NOUN$) = 0.4$
*   $P($flies$|$VERB$) = 0.4$
*   $P($like$|$VERB$) = 0.1$
*   $P($like$|$PREP$) = 0.3$
*   $P($an$|$DET$) = 0.6$
*   $P($arrow$|$NOUN$) = 0.4$

Next, we draw two tables called **viterbi** and **backpointer** of size (4, 5). (There are 4 unique POS tags in this tagset and there are 5 words in the observed sentence)

**viterbi:**

Tag\Sent | ***time*** | ***flies*** | ***like*** | ***an*** | ***arrow***
--- | --- | --- | --- | --- | ---
DET | | | | | 
NOUN | | | | | 
VERB | | | | | 
PREP | | | | | 

**backpointer:**

Tag\Sent | ***time*** | ***flies*** | ***like*** | ***an*** | ***arrow***
--- | --- | --- | --- | --- | ---
DET | | | | | 
NOUN | | | | | 
VERB | | | | | 
PREP | | | | | 

A cell, say corresponding to (***like***, VERB), in the ***viterbi*** table stores the maximum probability of all the sequences of tags until that point from left to right. For example,

**viterbi**(***like***, VERB) $= \underset{(t_1, t_2, \text{VERB}) \in \mathbb{T}_n}{\operatorname{max}}P(t_1)P($***time***$|t_1)P(t_2|t_1)P($***flies***$|t_2)P($VERB$|t_2)P($like$|$VERB$)$

The same cell in the **backpointer** table stores the tag of the previous word from which the maximum viterbi score was arrived at. For example, if

($\tilde{t_1}$, $\tilde{t_2}$, VERB) $= \underset{(t_1, t_2, \text{VERB}) \in \mathbb{T}_n}{\operatorname{argmax}}P(t_1)P($***time***$|t_1)P(t_2|t_1)P($***flies***$|t_2)P($VERB$|t_2)P($like$|$VERB$)$

then, **backpointer**(***like***, VERB) = $\tilde{t_2}$

**Q12: Let's begin by filling the first column of these tables.**

As an example, **viterbi**(***time***, NOUN) = $\underset{(\text{NOUN}) \in \mathbb{T}_1}{\operatorname{max}}P($NOUN$)P($***time***$|$NOUN) = $ P($NOUN$|start)P($***time***$|$NOUN) = $0.5 \times 0.7$

What is **viterbi**(***time***, VERB)?
What is the first column of the **backpointer** table? 

**viterbi:**

Tag\Sent | ***time*** | ***flies*** | ***like*** | ***an*** | ***arrow***
--- | --- | --- | --- | --- | ---
DET | ? | | | | 
NOUN |0.35 | | | | 
VERB | ? | | | | 
PREP | ? | | | | 

**backpointer:**

Tag\Sent | ***time*** | ***flies*** | ***like*** | ***an*** | ***arrow***
--- | --- | --- | --- | --- | ---
DET | - | | | | 
NOUN | start | | | | 
VERB | start | | | | 
PREP | - | | | | 

**Now we fill the second column.**
As an example, 

**viterbi**(***flies***, NOUN) = $\underset{(t_1, \text{NOUN}) \in \mathbb{T}_1}{\operatorname{max}}P(t_1)P($***time***$|t_1)P($NOUN$|t_1)P($***flies***$|$NOUN) = $ \underset{(t_1, \text{NOUN}) \in \mathbb{T}_1}{\operatorname{max}}$**viterbi**(***time***, $t_1$)$P($NOUN$|t_1)P($***flies***$|$NOUN)

So all you need is the first column of the viterbi table and transition probabilites and the emission probabilities to compute the entries of the second column.

**viterbi**(***time***, NOUN)$P($NOUN$|$NOUN$)P($***flies***$|$NOUN) = $0.35 \times 0.2 \times 0.4 = 0.028$

**Q13:** Compute **viterbi**(***time***, VERB)$P($NOUN$|$VERB$)P($***flies***$|$NOUN). Is this greater than $0.028$? If yes, then enter the new computed value in the cell **viterbi**(***flies***, NOUN) and enter VERB in **backpointer**(***flies***, NOUN). If no, then enter $0.028$ in **viterbi**(***flies***, NOUN) and enter NOUN in **backpointer**(***flies***, NOUN). Do this for every cell in the second column.


**viterbi:**

Tag\Sent | ***time*** | ***flies*** | ***like*** | ***an*** | ***arrow***
--- | --- | --- | --- | --- | ---
DET | ? | | | | 
NOUN |0.35 | 0.028 or ? | | | 
VERB | ? | | | | 
PREP | ? | | | | 

**backpointer:**

Tag\Sent | ***time*** | ***flies*** | ***like*** | ***an*** | ***arrow***
--- | --- | --- | --- | --- | ---
DET | - | | | | 
NOUN | start | NOUN or VERB?| | | 
VERB | start | | | | 
PREP | - | | | | 

**Q14:** Now fill the third column using entries in the second column and the transition probabilities and emission probabilites provided.

For your reference, the formulation is as follows:

**viterbi**(***like***, VERB) = $ \underset{(t_1, t_2, \text{VERB}) \in \mathbb{T}_1}{\operatorname{max}}$**viterbi**(***flies***, $t_2$)$P($VERB$|t_2)P($***like***$|$NOUN)

And the **backpointer**(***like***, VERB) will be that tag $t_2$ which resulted in the maximum viterbi score of **viterbi**(***like***, VERB)

**Q15:** Column by column fill the entire **viterbi** and **backpointer** tables below.

**viterbi:**

Tag\Sent | ***time*** | ***flies*** | ***like*** | ***an*** | ***arrow***
--- | --- | --- | --- | --- | ---
DET | ? | ?| ? | ? | ? 
NOUN | ? | ? | ? | ? | ?
VERB | ? | ? | ? | ? | ?
PREP | ? | ? | ? | ? | ?

**backpointer:**

Tag\Sent | ***time*** | ***flies*** | ***like*** | ***an*** | ***arrow***
--- | --- | --- | --- | --- | ---
DET | ? | ? | ?| ?| ?
NOUN | ? | ?| ?| ?| ?
VERB | ? | ?| ?| ?| ?
PREP | ? | ?| ?| ?| ? 

**Q16:** In the last column, **viterbi**(***arrow***, tag), find the tag $\tilde{t}$ which has the greatest value. Select the corresponding cell in the other table **backpointer**(***arrow***, $\tilde{t}$) and trace back the path to get the solution. What is the solution $\tilde{T}$ sequence of tags according to the provided HMM model? 

In [ ]:
# ENTER YOUR ANSWER HERE

**Optional Q:** What is the time-complexity of this Viterbi algorithm in terms of length of observed sentence and size of the tagset?

In [ ]:
# ENTER YOUR ANSWER HERE

# 5. Computing transition probabilities and emission probabilities 

We will now compute the following from the training set,

1.   Initial state probabilities ```p_initial```
2.   Last state probabilities ```p_last``` (optional)
3.   State transition probabilities ```p_transition```
4.   Emission probabilities ```p_emission```

Recall from Q11, how can we estimate these probabilities using counts from the training set?

**Q17:** We will first compute the initial state probabilities for all tags and store it in ```p_initial``` dictionary. In other words, the probability of the first state/POS_tag is a ```'NOUN'``` is stored in ```p_initial['NOUN']```. The simple formula we use is:

$P($NOUN$|start) = $ count(NOUN in first position in training set ) / size of training set

What is the most probable and the least probable POS Tag of the first/initial state? (Sanity check: ```p_initial['PRON']``` should be 15.9%)

In [ ]:
##########
p_initial = {}
## ENTER CODE: populate the above dictionary

##########

sorted_p_initial = sorted(p_initial.items(), key=operator.itemgetter(1))

print(p_initial['PRON'])  # Sanity check
print('Initial probabilities are', p_initial)
print('The most probable tag is', sorted_p_initial[-1])
print('The least probable tag is', sorted_p_initial[0])

**Q18 (optional):** Similarly, we can also compute the last state probabilities and store it in ```p_last``` dictionary. In other words, the probability of the last state/POS_tag is a 'NOUN' is stored in p_last['NOUN'].

What is the most probable and the least probable POS tag of the last state? (Sanity check: ```p_initial['PRON']``` should be $9.3738 \times 10^{-5}$)

NOTE: In the current HMM formulation, we don't require ```p_last```. However, we could use it in another HMM formulation where it is an additional term. The same viterbi algorithm could be applied to find the tags of an observed sequence of words/tokens in that case.

In [ ]:
##########
p_last = {}
## ENTER CODE: populate the above dictionary

##########

sorted_p_last = sorted(p_last.items(), key=operator.itemgetter(1))

print(p_last['PRON'])  # Sanity check
print('Last state probabilities are', p_last)
print('The most probable tag is', sorted_p_last[-1])
print('The least probable tag is', sorted_p_last[0])

**Q19:** To compute state transition probabilities, we first need to count the transitions and store it in a ```count_tag_tag``` dictionary of dictionaries. In other words, how often do we see a transition from ```NOUN``` to ```VERB``` in the training set will be stored in ```count_tag_tag['VERB']['NOUN']```. ***Should be read as count of Verb given Noun.***

Count the transition counts in ```count_tag_tag``` dictionary of dictionaries. (Sanity check - ```count_tag_tag['VERB']['NOUN']``` should be 40760 )

In [ ]:
##########
count_tag_tag = {}
## ENTER CODE HERE

##########

print(count_tag_tag['VERB']['NOUN']) # Sanity check: count of transition from NOUN to VERB

Now we have all the ingredients we need to compute the state transition probabilities ```p_transition``` and the emission probabilities ```p_emission```.

As an example,

```p_transition['VERB']['NOUN'] = count_tag_tag['VERB']['NOUN'] / count_tag['NOUN']``` 

***Should be read as probability of Verb given Noun.***

**Q20:** Compute the state transition probabilities and store in ```p_transition``` dictionary of dictionaries which is of the same shape as ```count_tag_tag``` dictionary of dictionaries. (Sanity check - ```p_transition['VERB']['NOUN']``` should be 0.1588)

In [ ]:
p_transition = {}
for end_tag in count_tag_tag.keys():
  for start_tag in count_tag_tag[end_tag].keys():
    numerator = ## ENTER HERE
    denominator = ## ENTER HERE
    probability = float(numerator) / denominator
    if end_tag not in p_transition.keys():
      p_transition[end_tag] = {start_tag: probability}
    else:
      p_transition[end_tag][start_tag] = probability

print(p_transition['VERB']['NOUN'])
print(count_tag_tag['VERB']['NOUN'])
print(count_tag['VERB'])

**Q21:** Similarly, compute the emission probabilities and store in ```p_emission``` dictionary of dictionaries which is of the same shape as ```count_token_tag``` from Q7. (Sanity check - ```p_emission['the']['DET']```, which is probability of 'the' given DET, should be 0.4574)

In [ ]:
p_emission = {}
for _token in count_token_tag.keys():
  for _tag in count_token_tag[_token].keys():
    numerator = ## ENTER HERE
    denominator = ## ENTER HERE
    probability = float(numerator) / denominator
    if _token not in p_emission.keys():
      p_emission[_token] = {_tag: probability}
    else:
      p_emission[_token][_tag] = probability

print(p_emission['the']['DET'])
print(count_token_tag['the']['DET'])
print(count_tag['DET'])

# 6. Viterbi algorithm

Pseudo-code examples of the [Viterbi algorithm](https://en.wikipedia.org/wiki/Viterbi_algorithm) are available in the lecture slides and online.

We will create a function ```_HMMtagger(sentence)``` which takes an input sequence/tuple of words/tokens ```sentence``` and returns an output sequence/tuple of POS tags of the same size. This output is the most probable tag sequence of the input sentence according to our HMM model. We will implement the Viterbi algorithm to find the most probable tag sequence and use the global variable python dictionaries ```p_initial```, ```p_transition```, and ```p_emission``` from the previous section 5 to do so.

**Q22:** Complete the code below at specified places. Look out for ```## ENTER YOUR ANSWER HERE```.

In [ ]:
import numpy as np # for logarithms

def _HMMtagger(sentence):
  """Returns the most probable tag sequece of a sentence as per HMM probabilites"""
  # Creating viterbi and backpointer matrices. 
  # For simplicity, we create a list of dictionaries. Dictionaries appended later.  
  viterbi = [] 
  backpointer = []

  # Initialize 
  # Filling the first column which is the the first dictionary in the above lists
  viterbi.append({})
  backpointer.append({})
  first_word = sentence[0]
  if first_word in p_emission.keys():
    for tag in p_emission[first_word].keys():
      if tag in p_initial.keys():
        viterbi[0][tag] = np.log(p_initial[tag]) + np.log(p_emission[first_word][tag])  
        # we work with log to handle small numbers. multiplication becomes addition in log scale.
        backpointer[0][tag] = '<s>'
  else:
    # If first_word not in p_emission.keys() then its because its an unseen word.
    # For unseen words we assume it is equally likely to be emitted by any tag.
    # We could have included an UNKNOWN token in the training set 
    # but that is suppose to be done during pre-processing step.  
    # Hence, for now, we will ignore p_emission[first_word][tag] 
    # when computing viterbi[0][tag]
    for tag in p_initial.keys():
      viterbi[0][tag] = ## ENTER YOUR ANSWER HERE  
      backpointer[0][tag] = '<s>' 
  
  # Filling other columns
  for i in range(len(sentence)-1):
    time_step = i+1
    viterbi.append({})
    backpointer.append({})
    word = sentence[time_step]
    if word in p_emission.keys():
      for tag in p_emission[word].keys():
        viterbi[time_step][tag] = float("-inf")    # to be updated in the following loop
        backpointer[time_step][tag] = 'placeholder'  # to be updated in following loop
        for previous_tag in viterbi[time_step-1].keys():
          if previous_tag in p_transition[tag].keys():
            new_viterbi = ## ENTER YOUR ANSWER HERE
            if new_viterbi > viterbi[time_step][tag]:
              viterbi[time_step][tag] = new_viterbi
              backpointer[time_step][tag] = ## ENTER YOUR ANSWER HERE
    else:
      # Just like before, the word is not seen in the training set
      # So we assume it is equally likely to be emitted by any tag
      for tag in tagset:
        # tagset from Q1
        viterbi[time_step][tag] = float("-inf")    # to be updated in the following loop
        backpointer[time_step][tag] = 'placeholder'  # to be updated in following loop
        for previous_tag in viterbi[time_step-1].keys():
          if previous_tag in p_transition[tag].keys():
            new_viterbi = ## ENTER YOUR ANSWER HERE. ignore p_emission[word][tag]
            if new_viterbi > viterbi[time_step][tag]:
              viterbi[time_step][tag] = new_viterbi
              backpointer[time_step][tag] = ## ENTER YOUR ANSWER HERE
  
  # Retracing the most probable path and returning it
  solution = []
  final_time_step = len(sentence)-1
  final_probability = float("-inf")  # to be updated in the following loop
  final_tag = 'placeholder'  # to be updated in the following loop
  for tag in viterbi[final_time_step].keys():
    if viterbi[final_time_step][tag] > final_probability:
      final_probability = viterbi[final_time_step][tag]
      final_tag = tag
  
  while final_tag != '<s>':
    solution.append(final_tag)
    final_tag = ## ENTER YOUR ANSWER HERE 
    final_time_step = final_time_step - 1
  
  solution.reverse()

  return solution

print(_HMMtagger(['This', 'is', 'just', 'an', 'example', 'to', 'check', 'if', 'the', 'tagger', 'works', 'okay', 'or', 'not']), 'Sanity check')

**Q23:** Evaluate the HMM tagger against ```test_set``` and ```external_test_set```. How does it compare to the baseline naive models? 

In [ ]:
def _HMM(x):
  """Tags every sentence in x with the _HMMtagger"""
  Y = x.copy()
  for i in range(len(x)):
    Y[i] = _HMMtagger(x[i])
  return Y

_evaluate(_HMM(test_set_x), test_set_Y)
_evaluate(_HMM(external_test_set_x), external_test_set_Y)

# 7. (optional) RNN-based Tagging model

In the previous tutorial we explored a simple RNN-based Language Model which maximizes the likelihood of the next word given the history of previous words in the sentence. Pictorially, the model looks as follows:

![RNNLM](https://raw.githubusercontent.com/torch/torch.github.io/master/blog/_posts/images/rnnlm.png)

**Q24:** If we replace the target words (see the above picture) with the POS tags and then train a RNN model then we get an RNN-based POS Tagger. Can you implement such a tagger? How much does the accuracy change?

# Doubts? Questions? Need help with this tutorial?
Contact me at clala@imperial.ac.uk or chiraag.r.lala@gmail.com :)